In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
import time

In [31]:
# 웹드라이버 실행
driver = webdriver.Chrome()
# 창을 최대로 키우기
driver.maximize_window()
# 제품 목록 페이지 로드
url = 'https://www.lg.com/in/tv-soundbars/all-tv-soundbars/?ec_model_status_code=ACTIVE&sortCriteria=%40ec_spotlights_order_no+descending%2C%40ec_salable_status+descending%2C%40ec_pdp_view_count+descending'
driver.get(url)

# 페이지 로딩 대기
wait = WebDriverWait(driver, 10)

# 모든 제품 페이지에서 제품 링크 수집
product_links = []

while True:
    # 제품 링크 요소들을 찾음
    products = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.product-grid-item a')))
    
    for product in products:
        link = product.get_attribute('href')
        if link and link not in product_links:  # 링크가 존재하고 중복되지 않으면 추가
            product_links.append(link)
    
    # 다음 페이지 버튼 찾기
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, 'a.pagination-next')  # 다음 페이지 버튼에 대한 CSS 선택자
        if not next_button.is_enabled() or "disabled" in next_button.get_attribute("class"):
            break  # 더이상 다음 페이지가 없으면 루프 종료
        next_button.click()
        time.sleep(5)  # 페이지 로딩 대기
    except Exception as e:
        print("다음 페이지로 이동 중 문제가 발생했습니다:", e)
        break  # 예외 발생 시 루프 종료
        
print(f'총 {len(product_links)}개의 제품 링크를 수집했습니다.')

# 수집된 제품 링크 확인 (선택사항)
for link in product_links:
    print(link)

TimeoutException: Message: 


In [24]:
# 수집한 모든 제품 링크 순회 (tqdm 추가)
for link in tqdm(product_links, desc="Processing Products"):
    driver.get(link)
    time.sleep(3)  # 페이지 로딩 대기
    
    try:
        # 리뷰 탭 클릭
        review_tab = driver.find_element(By.XPATH, '//a[contains(text(), "Reviews")]')
        review_tab.click()
        time.sleep(3)  # 리뷰 탭 로딩 대기
        
        # 스크롤 내려서 Locale 필터 설정
        driver.execute_script("window.scrollTo(0, 600);")
        time.sleep(2)  # 스크롤 완료 대기
        
        # Locale 필터 열기
        locale_filter = driver.find_element(By.CSS_SELECTOR, 'select.locale-filter')  # 적절한 CSS 선택자 사용
        locale_filter.click()
        time.sleep(1)
        
        # Locale을 English (India)로 설정
        english_option = driver.find_element(By.XPATH, '//option[contains(text(), "English (India)")]')
        english_option.click()
        time.sleep(3)  # 필터 적용 대기
        
        # Sort by 필터 설정
        sort_filter = driver.find_element(By.CSS_SELECTOR, 'select.sort-filter')  # 적절한 CSS 선택자 사용
        sort_filter.click()
        time.sleep(1)
        
        # Sort by Lowest to Highest Ratings 설정
        lowest_to_highest = driver.find_element(By.XPATH, '//option[contains(text(), "Lowest to Highest Ratings")]')
        lowest_to_highest.click()
        time.sleep(3)  # 정렬 적용 대기

        # 리뷰 데이터 수집
        reviews = driver.find_elements(By.CSS_SELECTOR, 'div.review')  # 적절한 CSS 선택자 사용
        for review in reviews:
            review_text = review.text
            print(review_text)  # 리뷰 텍스트 출력 혹은 저장

    except Exception as e:
        print(f"Error processing {link}: {e}")
        continue

In [25]:
driver.quit()

Processing TV products: 0it [00:00, ?it/s]


In [46]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time

# 웹드라이버 실행
driver = webdriver.Chrome()

# 창을 최대로 키우기
driver.maximize_window()

# 제품 목록 페이지 로드
url = 'https://www.lg.com/in/tv-soundbars/all-tv-soundbars'
driver.get(url)

# 페이지 로딩 대기
wait = WebDriverWait(driver, 30)

# 페이지가 제대로 열렸는지 확인
print("페이지 제목: ", driver.title)

# 페이지를 아래로 스크롤하여 모든 요소가 로드되도록 유도
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(5)  # 스크롤 후 대기

# 모든 제품 페이지에서 제품 링크 수집
product_links = []
base_url = 'https://www.lg.com'  # 상대 경로를 절대 경로로 변환하기 위한 기본 URL

try:
    # 제품 링크 요소들을 기다리며 찾음
    products = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.model-name a')))
    
    if not products:
        print("제품을 찾지 못했습니다.")
    
    for product in products:
        relative_link = product.get_attribute('href')
        if relative_link:
            full_link = base_url + relative_link
            if full_link not in product_links:  # 링크가 중복되지 않으면 추가
                product_links.append(full_link)
    
except TimeoutException:
    print("TimeoutException이 발생했습니다. 제품 링크를 찾지 못했습니다.")

print(f'총 {len(product_links)}개의 제품 링크를 수집했습니다.')

# 수집된 제품 링크 확인
for link in product_links:
    print(link)

# 웹드라이버 종료
driver.quit()


페이지 제목:  All TVs&Soundbars : Find the Products You Need | LG IN
TimeoutException이 발생했습니다. 제품 링크를 찾지 못했습니다.
총 0개의 제품 링크를 수집했습니다.


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from tqdm import tqdm
import pandas as pd

# 웹드라이버 설정
driver = webdriver.Chrome()
driver.maximize_window()

# LG 사운드바 페이지로 이동
url = "https://www.lg.com/in/tv-soundbars/all-tv-soundbars/?ec_model_status_code=ACTIVE&sortCriteria=%40ec_spotlights_order_no+descending%2C%40ec_salable_status+descending%2C%40ec_pdp_view_count+descending"
driver.get(url)

# 페이지에서 수집한 리뷰 데이터를 저장할 리스트 초기화
all_reviews = []

# 다음 페이지가 없을 때까지 모든 제품의 리뷰 수집
while True:
    try:
        # 제품 목록 로드 대기
        product_container = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.content-container"))
        )

        # 모든 제품 요소 수집
        products = product_container.find_elements(By.CSS_SELECTOR, "div.model-cell")
        
        if not products:
            print("No products found on this page.")
            break
        
        for product in tqdm(products, desc="Processing products"):
            try:
                # 제품 클릭하여 상세 페이지로 이동
                product.click()
                
                # Review 탭이 로드될 때까지 대기
                WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.LINK_TEXT, "Reviews"))
                ).click()
                
                # 스크롤을 내려서 Filter Reviews 표시
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight/3);")

                # Locale을 English(india)로 설정
                locale_dropdown = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "div#locale-select select"))
                )
                locale_dropdown.click()
                locale_option = driver.find_element(By.XPATH, "//option[@value='en_IN']")
                locale_option.click()

                # Sort by Lowest to Highest Ratings로 설정
                sort_dropdown = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "select#sort-options"))
                )
                sort_dropdown.click()
                lowest_ratings_option = driver.find_element(By.XPATH, "//option[@value='LOW_TO_HIGH']")
                lowest_ratings_option.click()

                # 모든 리뷰 수집
                reviews = driver.find_elements(By.CSS_SELECTOR, "div.review")
                for review in reviews:
                    rating = review.find_element(By.CSS_SELECTOR, "span.rating").text
                    review_text = review.find_element(By.CSS_SELECTOR, "p.description").text
                    all_reviews.append({
                        "product_name": driver.find_element(By.CSS_SELECTOR, "h1").text,
                        "rating": rating,
                        "review_text": review_text
                    })

            except Exception as e:
                print(f"Error processing product: {e}")
            
            finally:
                # 상세 페이지에서 제품 목록 페이지로 돌아가기
                driver.back()
                time.sleep(2)

    except Exception as e:
        print(f"An error occurred: {e}")
        break

    # 다음 페이지로 이동 시도
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, "a.next.page")
        next_button.click()
        time.sleep(3)
    except:
        print("No more pages.")
        break

# 수집된 리뷰 데이터를 Pandas DataFrame으로 변환
df = pd.DataFrame(all_reviews)

# 수집한 데이터를 저장
df.to_csv("lg_soundbar_reviews.csv", index=False)

print("Review collection complete. Data saved to lg_soundbar_reviews.csv")

An error occurred: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF673DE9642+30946]
	(No symbol) [0x00007FF673D9E3D9]
	(No symbol) [0x00007FF673C96FDA]
	(No symbol) [0x00007FF673CE822C]
	(No symbol) [0x00007FF673CE850C]
	(No symbol) [0x00007FF673D2DCB7]
	(No symbol) [0x00007FF673D0CAAF]
	(No symbol) [0x00007FF673D2B041]
	(No symbol) [0x00007FF673D0C813]
	(No symbol) [0x00007FF673CDA6E5]
	(No symbol) [0x00007FF673CDB021]
	GetHandleVerifier [0x00007FF673F1F84D+1301229]
	GetHandleVerifier [0x00007FF673F2BDC7+1351783]
	GetHandleVerifier [0x00007FF673F22A13+1313971]
	GetHandleVerifier [0x00007FF673E1DD16+245686]
	(No symbol) [0x00007FF673DA759F]
	(No symbol) [0x00007FF673DA3814]
	(No symbol) [0x00007FF673DA39A2]
	(No symbol) [0x00007FF673D9A3FF]
	BaseThreadInitThunk [0x00007FFA985B257D+29]
	RtlUserThreadStart [0x00007FFA99D0AF28+40]

Review collection complete. Data saved to lg_soundbar_reviews.csv


In [ ]:
# 웹드라이버 종료
driver.quit()